**Instrucciones:**
- Ejecutar para ambas BD (SQL y NoSQL) las siguentes consultas (queries):
    - Encontrar el valor mínimo de la lóngitud del sépalo
    - Encontrar el valor máximo del ancho del pétalo
    - Encontrar el promedio del ancho del sépalo
    - Encontrar la desviación estándar de la longitud del pétalo
    - ¿Cuál es el rango (max-min) de todos los atributos (Ancho/longitud del pétalo/sépalo)?

In [ ]:
# Es posible que tengamos que instalar el controlador MongoDB
!apt install mongodb
!service mongodb start

In [ ]:
import json
from pymongo import MongoClient 
  
  
# Establecer una concexión
myclient = MongoClient() 
   
# database 
db = myclient["BDEjemplo"]
   
# Crear una colección
Collection = db["iris_data"]
  
# Abrimos el archivo Json
with open('iris.json') as file:
    file_data = json.load(file)
      
# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else inser_one is used
if isinstance(file_data, list):
    Collection.insert_many(file_data)  
else:
    Collection.insert_one(file_data)

In [ ]:
# Encontrar el valor mínimo para la longitud del sépalo
res = Collection.find(projection={"_id": False, "sepalLength":True}).sort("sepalLength").limit(1)

for x in res:
  print(x)

{'sepalLength': 4.3}


In [ ]:
type(res)

pymongo.cursor.Cursor

In [ ]:
# Encontrar el valor máximo para el ancho del pétalo - Opción 1
resultado = Collection.find(projection={"_id":False,"petalWidth":True}).sort("petalWidth",-1).limit(1)

for x in resultado:
  print(x)

{'petalWidth': 2.5}


In [ ]:
resultado = Collection.find(projection={"_id":False,"petalWidth":True}).sort()

In [ ]:
# Encontrar el valor máximo para el ancho del pétalo - Opción 2
import numpy as np
res = Collection.find(projection={"_id": False, "petalWidth":True})

lista = []
for x in res:
  lista.append(x['petalWidth'])

arr = np.array(lista)
print(arr.max())

2.5


In [ ]:
# No existe el método max en una lista por eso usamos el arreglo de numpy
lista.max()

In [ ]:
# Encontrar el valor máximo para el ancho del pétalo - Opción 3
agg_result = Collection.aggregate(
 [
    {
         "$group":
       {

            "_id":"_id",
           "Máximo": { "$max": "$petalWidth" }
        }
    }
  ]
)

for i in agg_result:
    print(i)

{'_id': '_id', 'Máximo': 2.5}


In [ ]:
# Encontrar el valor máximo para el ancho del pétalo - Opción 3
agg_result = Collection.aggregate(
 [
    {
         "$addFields":
       {

            "_id":"_id",
           "Máximo": { "$max": "$petalWidth" }
        }
      }
  ]
)

for i in agg_result:
    print(i)

In [ ]:
# Para revisar la versión de pymongo
import pymongo
print(pymongo.__version__)

3.11.4


**Agregaciones**

Las operaciones de agregación procesan los registros de datos y devuelven los resultados *calculados*.
- Las operaciones de agregación agrupan valores de varios documentos y pueden realizar una variedad de operaciones en los datos agrupados para devolver un único resultado.
- MongoDB proporciona tres formas de realizar la agregación: 
  - Aggregation Pipeline (Flujo de agregación).
  - la función de reducción de mapa.
  - Single Purpose Aggregation Operations (Métodos o funciones de agregación de propósito único).

**Aggregation Pipeline**

El *framework* de agregación de MongoDB se basa en el concepto de *pipelines* de procesamiento de datos.

**Pipeline Expressions**

Algunas etapas de canalización toman una expresión de canalización como operando. Las expresiones de canalización especifican la transformación que se aplicará a los documentos de entrada. Las expresiones tienen una estructura de documento y pueden contener otras expresiones.

**Recursos**
- [Documentación](https://docs.mongodb.com/manual/aggregation/)
- [Stages (etapas) del pipeline (flujo)](https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/#std-label-aggregation-pipeline-operator-reference)
- [Libro: Practical MongoDB Aggregations](https://www.practical-mongodb-aggregations.com/)

In [ ]:
# Encontrar el promedio del ancho del sépalo - Opción 1
import numpy as np
res = Collection.find(projection={"_id": False, "sepalWidth":True})

lista = []
for x in res:
  lista.append(x['sepalWidth'])

arr = np.array(lista)
print(arr.mean())

3.0573333333333337


In [ ]:
# Encontrar el promedio del ancho del sépalo - Opción 2
res = Collection.find(projection={"_id": False, "sepalWidth":True})

suma = 0
numDocs = 0
for x in res:
  suma += x['sepalWidth'] # suma = suma + x['sepalWidth']
  numDocs += 1 # numDocs = numDocs + 1

promedio = suma/numDocs
print(promedio)

3.057333333333334


In [ ]:
# Encontrar el promedio del ancho del sépalo - Opción 3
agg_result = Collection.aggregate(
 [
    {
         "$group":
       {

            "_id":"_id",
           "AverageValue": { "$avg": "$sepalWidth" }
        }
      }
  ]
)

for i in agg_result:
    print(i)

{'_id': '_id', 'AverageValue': 3.0573333333333337}


In [ ]:
# Encontrar la desviación estándar de la longitud del pétalo - Opción 1
import numpy as np
res = Collection.find(projection={"_id": False, "petalLength":True})

lista = []
for x in res:
  lista.append(x['petalLength'])

arr = np.array(lista)
print(arr.std()) #1.7652982332594664

1.759404065775303


In [ ]:
# Encontrar la desviación estándar de la longitud del pétalo - Opción 2
import math
res = Collection.find(projection={"_id": False, "petalLength":True})

suma = 0
numDocs = 0
for x in res:
  suma += x['petalLength'] # suma = suma + x['petalLength']
  numDocs += 1 # numDocs = numDocs + 1
promedio = suma/numDocs

res = Collection.find(projection={"_id": False, "petalLength":True})
suma = 0
for x in res:
  suma += (x['petalLength'] - promedio)**2 # suma = suma + x['petalLength']

std = math.sqrt(suma/numDocs)

print(std)

1.7594040657753032


In [ ]:
type(res)

list

In [ ]:
# Encontrar la desviación estándar de la longitud del pétalo - Opción 2
import math
res = list(Collection.find(projection={"_id": False, "petalLength":True}))

suma = 0
numDocs = 0
for x in res:
  suma += x['petalLength'] # suma = suma + x['petalLength']
  numDocs += 1 # numDocs = numDocs + 1
promedio = suma/numDocs

suma = 0
for x in res:
  suma += (x['petalLength'] - promedio)**2 # suma = suma + x['petalLength']

# Muestral
std = math.sqrt(suma/(numDocs-1))

# Poblacional
#std = math.sqrt(suma/(numDocs))

print(std)

1.7652982332594667


In [ ]:
len(list(res))

0

In [ ]:
# ¿POR QUÉ TENGO QUE EJECUTAR OTRO "FIND" PARA OTRO CICLO FOR?
import math
res = [{"v":1},{"v":2},{"v":3},{"v":4},{"v":5}]

suma = 0
numDocs = 0
for x in res:
  suma += x["v"] # suma = suma + x['petalLength']
  numDocs += 1 # numDocs = numDocs + 1
promedio = suma/numDocs

suma = 0
for x in res:
  suma += (x["v"]- promedio)**2 # suma = suma + x['petalLength']

std = math.sqrt(suma/numDocs)

print(std)

1.4142135623730951


In [ ]:
# Encontrar la desviación estándar de la longitud del pétalo - Opción 3
agg_result = Collection.aggregate(
 [
    {
         "$group":
       {

            "_id":"_id",
           "std": { "$stdDevPop": "$petalLength" }
        }
      }
  ]
)

for i in agg_result:
    print(i)     # 1.759404065775303

{'_id': '_id', 'std': 1.759404065775303}


In [ ]:
# ¿Cuál es el rango (max-min) de todos los atributos (Ancho/longitud del pétalo/sépalo)?
agg_result = Collection.aggregate(
 [
    {
         "$group":{
           "_id":"_id",
           "Mínimo": { "$min": "$sepalLength" },
           "Máximo": { "$max": "$sepalLength" },
           "AverageValue": { "$avg": "$sepalWidth" },
           "std": { "$stdDevPop": "$petalLength" },
        }
    },
    {
        "$addFields" : {
            "rango": { "$subtract": ["$Máximo", "$Mínimo"] }
        }        
    }
  ]
)

for i in agg_result:
    print(i)

{'_id': '_id', 'Mínimo': 4.3, 'Máximo': 7.9, 'AverageValue': 3.0573333333333337, 'std': 1.759404065775303, 'rango': 3.6000000000000005}


In [ ]:
# ¿Cuál es el rango (max-min) de todos los atributos (Ancho/longitud del pétalo/sépalo)?
agg_result = Collection.aggregate(
 [
    {
        "$group" : {
            "_id":"_id",
            "rango": { "$subtract": [{ "$max": "$sepalLength" }, { "$min": "$sepalLength" }] }
        }        
    }
  ]
)

for i in agg_result:
    print(i)

In [ ]:
#'Mínimo': 4.3, 'Máximo': 7.9
7.9-4.3

3.6000000000000005